# Homework 2

## 2.1 Spam Filter

In [ ]:
spam_corpus = [["I", "am", "spam", "spam", "I", "am"], ["I", "do", "not","like", "that", "spamiam"]]
ham_corpus = [["do", "i", "like", "green", "eggs", "and", "ham"], ["i", "do"]]

def createWordList(emails):
    wordList = []
    for email in emails:
        for word in email:
            wordList.append(word)
    return wordList

def createDict(corpus):
    result = dict()
    for email in corpus:
        for word in email:
            if (word in result.keys()):
                result[word] = result[word] + 1
            else:
                result[word] = 1
    return result

def wordSearch(ourDict, word):
    if (word not in ourDict.keys()):
        return 0
    else:
        return ourDict[word]

# use Paul Graham's algorithm to find probability of email being spam
def findSpamProbability(word):
    totalReal = len(ham_corpus)
    totalSpam = len(spam_corpus)
    real = 2 * wordSearch(realDict, word)
    spam = 2 * wordSearch(spamDict, word)
    if (real + spam > 1):
        result = 0
        minSpam = min(1, spam / totalSpam)
        minTotal = min(1, real / totalReal) + minSpam
        percentageSpam = minSpam / minTotal
        spamProbbability = min(0.99, percentageSpam)
        result = max(0.01, spamProbability)
    else:
        result = 0
    return result
    
# build probability hash table
def createProbabilityHash(email):
    result = {}
    for word in email:
        wordLower = word.lower()
        result[word] = findSpamProbability(word)
    return result

realDict = createDict(ham_corpus)
spamDict = createDict(spam_corpus)

testEmail = createWordList(ham_corpus + spam_corpus)
probabilityDict = createProbabilityHash(email)

This is Bayesian probability because it takes into account the previous answer when calculating the probablities of the following cases. If each one was independent, it would not be Bayesian.

## 2.2 Lawn Care

In [ ]:
import sys
sys.path.insert(0, '../tools/aima')
from probability import BayesNet, enumeration_ask

# utility variables
T, F = True, False

# a) implement the network using the AIMA Python tools
grass = BayesNet([
    ('Cloudy', '', 0.50),
    ('Sprinkler', 'Cloudy', {T: 0.10, F: 0.50}),
    ('Rain', 'Cloudy', {T: 0.80, F: 0.20}),
    ('WetGrass', 'Sprinkler Rain', {(T, T): 0.99, (T, F): 0.90, (F, T): 0.90, (F, F): 0.00})
])

# b) there are 16 independent values in the full joint distribution of this domain because it 
# is 2 ^ (number of variables). in this case, we have 4 variables, and therefore there are 16 
# independent values.

# c) by looking at the bayesian network, we can see that there's only one completely 
# independent variable, and it's the Cloudy variable. it doesn't have any parent variables and 
# all of the other variables are dependent on it.

# d) compute a bunch of things based on the network. work by hand will be at the end
print('i)   P(Cloudy): ', enumeration_ask('Cloudy', dict(), grass).show_approx())
print('ii)  P(Sprinkler | Cloudy): ', enumeration_ask('Sprinkler', dict(Cloudy=T), grass).show_approx())
print('iii) P(Cloudy | Sprinkler && No Rain): ', enumeration_ask('Cloudy', dict(Sprinkler=T, Rain=F), grass).show_approx())
print('iv)  P(Wet Grass | Cloudy && Sprinkler && Rain): ', enumeration_ask('WetGrass', dict(Cloudy=T, Sprinkler=T, Rain=T), grass).show_approx())
print('v)   P(Cloudy | Dry Grass): ', enumeration_ask('Cloudy', dict(WetGrass=F), wetGrassNetwork).show_approx())

# i)   P(Cloudy) = < P(Cloudy), P(Not Cloudy) >
#                = < 0.50, 0.50 >
# ii)  P(Sprinkler | Cloudy) = < P(Sprinkler | Cloudy), P(No Sprinkler | Cloudy) >
#                            = < 0.10, 0.90 >
# iii) P(Cloudy | Sprinkler && No Rain) = a < P(Cloudy | Sprinkler && No Rain), P(Not Cloudy | Sprinkler && No Rain) >
#                                       = a < P(Cloudy) * P(Cloudy & Sprinkler) * P(Cloudy & No Rain), 
#                                             P(Not Cloudy) * P(Not Cloudy & Sprinkler) * P(Not Cloudy & No Rain) >
#                                       = a < 0.50 * 0.10 * 0.20, 0.50 * 0.50 * 0.80 >
#                                       = a < 0.01, 0.20 >
#                                       = < 0.0476, 0.952 >
# iv)  P(Wet Grass | Cloudy && Sprinkler && Rain) = a < P(Wet Grass | Cloudy && Sprinkler && Rain), 
#                                                       P(Dry Grass | Cloudy && Sprinkler && Rain) >
#                                                 = a < P(Wet Grass & Cloudy & Sprinkler & Rain),
#                                                       P(Dry Grass & Cloudy & Sprinkler & Rain) >
#                                                 = a < 0.99 * 0.50 * 0.10 * 0.80, 0.01 * 0.50 * 0.10 * 0.80 >
#                                                 = a < 0.0396, 0.0004 >
#                                                 = < 0.99, 0.01 >
# v)   P(Cloudy | Dry Grass) = a < P(Cloudy | Dry Grass), P(Not Cloudy, Dry Grass) >
#                            = a < 0.50 * ((0.10 * 0.80 * 0.99) + (0.90 * 0.80 * 0.90) + (0.10 * 0.20 * 0.90)),
#                                  (0.50 * 0.50 * 0.80) + (0.50 * (0.50 * 0.20 * 0.01) + (0.50 * 0.80 * 0.10) + 
#                                  (0.50 * 0.20 * 0.10)) >
#                            = a < 0.3726, 0.2255 >
#                            = < 0.639, 0.361 >